In [1]:
import matplotlib.pyplot as plt
from pyomo.environ import *
from pyomo.dae import *

import numpy as np
from scipy.interpolate import interp2d
import pandas as pd 
from itertools import permutations, product, combinations

from fim_doe import *
from pyomo.contrib.sensitivity_toolbox.sens import sipopt

## Measurements class

In [2]:
# generate a set of measurements with extra index CA, CB, CC
# each extra index has different measure time points
t_measure_ca = [0, 0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.875, 1]
t_measure_cb = [0, 0.25, 0.5, 0.75, 1]
t_measure_cc = [0, 0.125, 0.375, 0.625, 0.875, 1]
measure_pass = {'C':{"CA": t_measure_ca, "CB": t_measure_cb, "CC": t_measure_cc}}


In [3]:
measure_class = Measurements(measure_pass)


All measurements are flattened.
Flatten measurement name: ['C_index_CA', 'C_index_CB', 'C_index_CC']
Flatten measurement timeset: {'C_index_CA': [0, 0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.875, 1], 'C_index_CB': [0, 0.25, 0.5, 0.75, 1], 'C_index_CC': [0, 0.125, 0.375, 0.625, 0.875, 1]}


In [4]:
# a legal subset
subset_1 = {'C':{"CA": t_measure_cb}}
measure_subclass1 = Measurements(subset_1)
measure_class.check_subset(measure_subclass1)

# an illegal subset with wrong time
# Expect error: ('The time of ', 0.125, ' is not included as measurements before.')
subset_2 = {'C':{"CB": t_measure_cc}}
measure_subclass2 = Measurements(subset_2)
measure_class.check_subset(measure_subclass2)

All measurements are flattened.
Flatten measurement name: ['C_index_CA']
Flatten measurement timeset: {'C_index_CA': [0, 0.25, 0.5, 0.75, 1]}
All measurements are flattened.
Flatten measurement name: ['C_index_CB']
Flatten measurement timeset: {'C_index_CB': [0, 0.125, 0.375, 0.625, 0.875, 1]}


ValueError: ('The time of ', 0.125, ' is not included as measurements before.')

## Scenario class

In [5]:
param_dict = {'A1': 84.79, 'A2': 371.72, 'E1': 7.78, 'E2': 15.05}

scenario_gene = Scenario_generator(param_dict)
simultaneous_set = scenario_gene.simultaneous_scenario()
print(simultaneous_set)

{'A1': {0: 84.87478999999999, 1: 84.79, 2: 84.79, 3: 84.79, 4: 84.70521000000001, 5: 84.79, 6: 84.79, 7: 84.79}, 'A2': {0: 371.72, 1: 372.09172, 2: 371.72, 3: 371.72, 4: 371.72, 5: 371.34828000000005, 6: 371.72, 7: 371.72}, 'E1': {0: 7.78, 1: 7.78, 2: 7.78778, 3: 7.78, 4: 7.78, 5: 7.78, 6: 7.77222, 7: 7.78}, 'E2': {0: 15.05, 1: 15.05, 2: 15.05, 3: 15.06505, 4: 15.05, 5: 15.05, 6: 15.05, 7: 15.03495}, 'jac-index': {'A1': [0, 4], 'A2': [1, 5], 'E1': [2, 6], 'E2': [3, 7]}, 'eps-abs': {'A1': 0.16958, 'A2': 0.7434400000000001, 'E1': 0.015560000000000001, 'E2': 0.030100000000000002}, 'scena-name': [0, 1, 2, 3, 4, 5, 6, 7]}


In [9]:
scenario_gene.generate_sequential_para()
print(scenario_gene.eps_abs)
print(scenario_gene.scenario_para)

for i in range(8):
    ss_next = scenario_gene.next_sequential_scenario(i)
    print(ss_next)

{'A1': 0.16958, 'A2': 0.7434400000000001, 'E1': 0.015560000000000001, 'E2': 0.030100000000000002}
{'A1': [0, 4], 'A2': [1, 5], 'E1': [2, 6], 'E2': [3, 7]}
{'A1': {0: 84.87478999999999}, 'A2': {0: 371.72}, 'E1': {0: 7.78}, 'E2': {0: 15.05}, 'jac-index': {'A1': [0], 'A2': [0], 'E1': [0], 'E2': [0]}, 'eps-abs': {'A1': 0.08487478999999999, 'A2': 0.37172000000000005, 'E1': 0.0077800000000000005, 'E2': 0.015050000000000001}, 'scena-name': [0]}
{'A1': {0: 84.79}, 'A2': {0: 372.09172}, 'E1': {0: 7.78}, 'E2': {0: 15.05}, 'jac-index': {'A1': [0], 'A2': [0], 'E1': [0], 'E2': [0]}, 'eps-abs': {'A1': 0.08479, 'A2': 0.37209172, 'E1': 0.0077800000000000005, 'E2': 0.015050000000000001}, 'scena-name': [0]}
{'A1': {0: 84.79}, 'A2': {0: 371.72}, 'E1': {0: 7.78778}, 'E2': {0: 15.05}, 'jac-index': {'A1': [0], 'A2': [0], 'E1': [0], 'E2': [0]}, 'eps-abs': {'A1': 0.08479, 'A2': 0.37172000000000005, 'E1': 0.00778778, 'E2': 0.015050000000000001}, 'scena-name': [0]}
{'A1': {0: 84.79}, 'A2': {0: 371.72}, 'E1': {0

## Regression tests